In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Database Reading and Manipulation
import pandas as pd

# Linear Algebra
import numpy as np

# Plotting
import matplotlib.pyplot as plt

# Processing results
import json

# Model Selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

# Modeling
from sklearn.linear_model import LinearRegression

# Processing
from sklearn.preprocessing import StandardScaler

# Data imputation
from sklearn.impute import SimpleImputer

# Pipeline
from sklearn.pipeline import Pipeline

# Metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score


# Custom modules
## Model selection
from src.cross_validation.blocking_time_series_split import BlockingTimeSeriesSplit

## Function to print scores
from src.utils.print_scores import print_scores

## Function to calculate score regression metrics
from src.utils.score_regression_metrics import score_regression_metrics

## Function to fill the results metric dict
from src.utils.fill_results_dict import fill_results_dict

<IPython.core.display.Javascript object>

# Functions and definitions

In [3]:
SEED = 47
METRICS = (
    "neg_root_mean_squared_error",
    "neg_mean_absolute_error",
    "neg_mean_absolute_percentage_error",
    "r2",
)
METRICS_DICT = {
    "neg_root_mean_squared_error": "RMSE",
    "neg_mean_absolute_error": "MAE",
    "neg_mean_absolute_percentage_error": "MAPE",
    "r2": "R2",
}

<IPython.core.display.Javascript object>

## Defining a dataframe structure to save the results

In [4]:
results_to_save = []

results_dict = {
    "Category": "Local Model",
    "Company": "partner_ii",
    "Features": "Chemical + Mineralogical",
    "Data Shape": None,
    "Timesteps": None,
    "Model": "Linear Regression",
    "Model Params": None,
    "Scaler": "Standard Scaler",
    "Scaler Params": None,
    "Imputer": "Median",
    "Imputer Params": None,
    "Cross Validation": None,
    "Cross Validation Params": np.nan,
    "RMSE Train": np.nan,
    "MAE Train": np.nan,
    "MAPE Train": np.nan,
    "R2 Train": np.nan,
    "RMSE Test": np.nan,
    "MAE Test": np.nan,
    "MAPE Test": np.nan,
    "R2 Test": np.nan,
}

<IPython.core.display.Javascript object>

# Reading the dataset

In [5]:
df = pd.read_csv("../../../../../../data/processed/partner_ii/cement-shipping.csv")

<IPython.core.display.Javascript object>

## Defining Features

In this set of experiments we keep only chemical and mineralogical features yielded by the same testing method/procedure

In [6]:
df_copy = df.drop(
    [
        # Properties
        
        
        "Initial setting time",
        "Blaine",
        "Sieve 32 um",
        "Sieve 45 um",
        "CS1",
        "CS3",
        "CS7",
        
        # Removing One-Hot encoding variables
        "Cement_Type",
        
    ],
    axis=1,
).copy()


<IPython.core.display.Javascript object>

# 1. Linear Regression

<h2>1. Dataset: df_no_cs</h2> <br>In this dataset the CS1, CS3  and CS7 variables are not considered. Only Chemical and mineralogical features measured by the same method. For this particular dataset, all chemical features, with the exception of LOI were measured by XRF and XRD methods.

In [7]:
y = df_copy.pop("CS28").values
x = df_copy.drop(["Date"], axis=1)
dates = df["Date"].copy()

<IPython.core.display.Javascript object>

## 1.1 Repeated KFold Cross validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Repeats:</b>10<br>
<b>Splits:</b>10<br>
    1. 10 folds of 123 samples each
    2. 90% train (1111 samples each fold)
    3. 10% test (123 samples each fold)
<b>Total:</b> 100 models<br>

In [8]:
repeats = 3
n_splits = 5
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)
cv = RepeatedKFold(n_splits=n_splits, n_repeats=repeats, random_state=SEED)
scores = cross_validate(
    pipeline,
    x,
    y,
    scoring=METRICS,
    cv=cv,
    n_jobs=-1,
    return_train_score=True,
)
print("Repeated Cross Validation:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Repeated KFold"
results_dict_copy["Cross Validation Params"] = '{"N_Splits": 5, "Repeats": 3}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(results_dict_copy, scores)
results_to_save.append(df_results)

Repeated Cross Validation:
Repeats: 3
n_splits: 5

******
[TRAIN]
******
RMSE: -2.410 (0.047)
MAE: -1.893 (0.027)
MAPE: -0.044 (0.001)
R2: 0.533 (0.014)


******
[TEST]
******
RMSE: -2.516 (0.234)
MAE: -1.939 (0.110)
MAPE: -0.045 (0.003)
R2: 0.484 (0.078)




<IPython.core.display.Javascript object>

In [9]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model Cross Validation RMSE Test  \
                                                                     mean   
0  Chemical + Mineralogical  Linear Regression   Repeated KFold  2.516459   

             MAE Test           MAPE Test             R2 Test            
        std      mean       std      mean       std      mean       std  
0  0.234451  1.938508  0.110079  0.045198  0.002606  0.483923  0.078293

<IPython.core.display.Javascript object>

## 1.2. Blocking Time Series Cross Validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Splits:</b>5<br>    
    1. 5 folds of 246 samples
    2. 50% train (123 samples each fold)
    3. 50% test (123 samples each fold)
<b>Total:</b> 5 models<br>

In [10]:
n_splits = 5
train_size = 0.8

pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)
cv = BlockingTimeSeriesSplit(n_splits=n_splits, train_size=train_size)
scores = cross_validate(
    pipeline,
    x,
    y,
    scoring=METRICS,
    cv=cv,
    n_jobs=-1,
    return_train_score=True,
)
print("Blocking Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Blocking Time Series Split"
results_dict_copy[
    "Cross Validation Params"
] = '{"N_Splits": 5, "Repeats": 1, "train_size": 0.8}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(results_dict_copy, scores)
results_to_save.append(df_results)

Blocking Time Series Split:
Repeats: 3
n_splits: 5

******
[TRAIN]
******
RMSE: -1.920 (0.182)
MAE: -1.524 (0.143)
MAPE: -0.036 (0.002)
R2: 0.616 (0.075)


******
[TEST]
******
RMSE: -3.312 (1.017)
MAE: -2.513 (0.681)
MAPE: -0.057 (0.013)
R2: -0.115 (0.475)




<IPython.core.display.Javascript object>

In [11]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model            Cross Validation  \
                                                                             
0  Chemical + Mineralogical  Linear Regression  Blocking Time Series Split   
1  Chemical + Mineralogical  Linear Regression              Repeated KFold   

  RMSE Test            MAE Test           MAPE Test             R2 Test  \
       mean       std      mean       std      mean       std      mean   
0  3.312442  1.016668  2.512650  0.680570  0.057328  0.013361 -0.114735   
1  2.516459  0.234451  1.938508  0.110079  0.045198  0.002606  0.483923   

             
        std  
0  0.475081  
1  0.078293

<IPython.core.display.Javascript object>

## 1.3. Time Series Split Cross Validation

The training set has size i * n_samples // (n_splits + 1) + n_samples % (n_splits + 1) in the i th split, with a test set of size n_samples//(n_splits + 1) by default, where n_samples is the number of samples.


<b>Dataset shape:</b> (1234, 38)<br>
<b>Splits:</b>10<br>    
    1. Train: 10 folds of 114, 226, 338, 450, 562, 675, 787, 899, 1011, 1123 samples each fold
    2. Test: 112 samples each fold
<b>Total:</b> 10 models<br>

In [12]:
n_splits = 5
gap = 0
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)
cv = TimeSeriesSplit(gap=gap, max_train_size=None, n_splits=n_splits, test_size=None)

scores = cross_validate(
    pipeline,
    x,
    y,
    scoring=METRICS,
    cv=cv,
    n_jobs=-1,
    return_train_score=True,
)
print("Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Time Series Split"
results_dict_copy["Cross Validation Params"] = '{"N_Splits": 5, "Repeats": 1, "Gap": 0}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(results_dict_copy, scores)
results_to_save.append(df_results)

Time Series Split:
Repeats: 3
n_splits: 5

******
[TRAIN]
******
RMSE: -2.234 (0.119)
MAE: -1.744 (0.119)
MAPE: -0.040 (0.003)
R2: 0.552 (0.067)


******
[TEST]
******
RMSE: -3.039 (0.705)
MAE: -2.409 (0.631)
MAPE: -0.057 (0.018)
R2: -0.101 (0.590)




<IPython.core.display.Javascript object>

In [13]:
pd.concat(results_to_save).groupby(["Features", "Model", "Cross Validation"])[
    ["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]
].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model            Cross Validation  \
                                                                             
0  Chemical + Mineralogical  Linear Regression  Blocking Time Series Split   
1  Chemical + Mineralogical  Linear Regression              Repeated KFold   
2  Chemical + Mineralogical  Linear Regression           Time Series Split   

  RMSE Test            MAE Test           MAPE Test             R2 Test  \
       mean       std      mean       std      mean       std      mean   
0  3.312442  1.016668  2.512650  0.680570  0.057328  0.013361 -0.114735   
1  2.516459  0.234451  1.938508  0.110079  0.045198  0.002606  0.483923   
2  3.038624  0.705377  2.408534  0.631157  0.057276  0.017876 -0.101039   

             
        std  
0  0.475081  
1  0.078293  
2  0.590346

<IPython.core.display.Javascript object>

## 1.4. Out of time Split Cross Validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Train size: 80%</b><br>
<b>Test  size: 20%</b>


<b>Splits:</b> 2<br>    
    1. Train: 987
    2. Test: 247
<b>Total:</b> 1 model<br>

In [14]:
test_size = 0.2

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=test_size, random_state=SEED, shuffle=False
)
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)

pipeline.fit(x_train, y_train)

y_train_pred = pipeline.predict(x_train)
y_test_pred = pipeline.predict(x_test)

scores = score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Out of time Split"
results_dict_copy["Cross Validation Params"] = '{"Test Size": 0.2}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(
    results_dict_copy, {key: [value] for key, value in scores.items()}
)
results_to_save.append(df_results)

******
[TRAIN]
******
RMSE: 2.304 (0.000)
MAE: 1.826 (0.000)
MAPE: 0.042 (0.000)
R2: 0.500 (0.000)


******
[TEST]
******
RMSE: 3.836 (0.000)
MAE: 3.253 (0.000)
MAPE: 0.083 (0.000)
R2: -0.900 (0.000)




<IPython.core.display.Javascript object>

In [15]:
pd.concat(results_to_save).groupby(["Features", "Model", "Cross Validation"])[
    ["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]
].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model            Cross Validation  \
                                                                             
0  Chemical + Mineralogical  Linear Regression  Blocking Time Series Split   
1  Chemical + Mineralogical  Linear Regression           Out of time Split   
2  Chemical + Mineralogical  Linear Regression              Repeated KFold   
3  Chemical + Mineralogical  Linear Regression           Time Series Split   

  RMSE Test            MAE Test           MAPE Test             R2 Test  \
       mean       std      mean       std      mean       std      mean   
0  3.312442  1.016668  2.512650  0.680570  0.057328  0.013361 -0.114735   
1  3.836344  0.000000  3.253284  0.000000  0.083310  0.000000 -0.899749   
2  2.516459  0.234451  1.938508  0.110079  0.045198  0.002606  0.483923   
3  3.038624  0.705377  2.408534  0.631157  0.057276  0.017876 -0.101039   

             
        std  
0  0.475081  
1  0.000000  
2  0.078293  
3  0.590346

<IPython.core.display.Javascript object>

In [16]:
pipeline.named_steps["estimator"].coef_

array([ 0.37239378, -0.84702317,  0.09664853, -0.00982781,  0.08686284,
        1.68111585,  0.26701594, -0.31242568,  2.17255686,  1.74390117,
       -0.24759652,  0.52026967,  0.34697121, -0.31769263])

<IPython.core.display.Javascript object>

In [17]:
pipeline.named_steps["estimator"].intercept_

43.77724465558196

<IPython.core.display.Javascript object>

In [18]:
coeff = pipeline.named_steps["estimator"].coef_
np.array(x.columns)[coeff == 0]

array([], dtype=object)

<IPython.core.display.Javascript object>

In [19]:
x.drop(np.array(x.columns)[coeff == 0], axis=1).columns

Index(['CaO', 'SiO2', 'Al2O3', 'Fe2O3', 'MgO', 'SO3', 'K2O', 'Na2O',
       'Total C3S', 'Total C2S', 'C3A', 'C4AF', 'Free CaO',
       'Loss on Ignition'],
      dtype='object')

<IPython.core.display.Javascript object>

In [20]:
x.drop(np.array(x.columns)[coeff == 0], axis=1).columns.shape

(14,)

<IPython.core.display.Javascript object>

In [21]:
x.columns.shape

(14,)

<IPython.core.display.Javascript object>

In [22]:
coeffs = pd.DataFrame(
    {col: [c] for col, c in zip(x.columns, coeff)}, index=["Coefficients"]
)

<IPython.core.display.Javascript object>

In [23]:
coeffs.T["Coefficients"].sort_values(ascending=False).to_frame(
    name="Coefficients"
).style.background_gradient(axis=None, vmin=1, vmax=5, cmap="Greens")

,Coefficients
Total C3S,2.172557
Total C2S,1.743901
SO3,1.681116
C4AF,0.520270
CaO,0.372394
Free CaO,0.346971
K2O,0.267016
Al2O3,0.096649
MgO,0.086863
Fe2O3,-0.009828


<IPython.core.display.Javascript object>

In [24]:
pd.concat(results_to_save).groupby(
    ["Features", "Model", "Cross Validation", "Cross Validation Params"]
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model            Cross Validation  \
                                                                             
0  Chemical + Mineralogical  Linear Regression  Blocking Time Series Split   
1  Chemical + Mineralogical  Linear Regression           Out of time Split   
2  Chemical + Mineralogical  Linear Regression              Repeated KFold   
3  Chemical + Mineralogical  Linear Regression           Time Series Split   

                            Cross Validation Params RMSE Test            \
                                                         mean       std   
0  {"N_Splits": 5, "Repeats": 1, "train_size": 0.8}  3.312442  1.016668   
1                                {"Test Size": 0.2}  3.836344  0.000000   
2                     {"N_Splits": 5, "Repeats": 3}  2.516459  0.234451   
3           {"N_Splits": 5, "Repeats": 1, "Gap": 0}  3.038624  0.705377   

   MAE Test           MAPE Test             R2 Test            
       mean       std      mean       std      mean       std  
0  2.512650  0.680570  0.057328  0.013361 -0.114735  0.475081  
1  3.253284  0.000000  0.083310  0.000000 -0.899749  0.000000  
2  1.938508  0.110079  0.045198  0.002606  0.483923  0.078293  
3  2.408534  0.631157  0.057276  0.017876 -0.101039  0.590346

<IPython.core.display.Javascript object>

# Saving the results Dataframe

## Saving the full dataframe

In [25]:
path = "../../../../../../reports/results/local_models/partner_ii/all_cements/full/"
filename = "linear_regression_results_full_3.csv"

pd.concat(results_to_save).to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>

## Saving the grouped dataframe

In [26]:
cols_groupby = [
    "Category",
    "Company",
    "Data Shape",
    "Timesteps",
    "Features",
    "Model",
    "Cross Validation",
    "Cross Validation Params",
]

cols_agg = ["RMSE Train", "MAE Train", "MAPE Train", "R2 Train"] + [
    "RMSE Test",
    "MAE Test",
    "MAPE Test",
    "R2 Test",
]

path = "../../../../../../reports/results/local_models/partner_ii/all_cements/grouped/"
filename = "linear_regression_results_grouped_3.csv"


df_results_to_save = (
    pd.concat(results_to_save)
    .groupby(cols_groupby, dropna=False)[cols_agg]
    .agg(["mean", lambda series: pd.Series(series.std(ddof=0), name="std")])
    .reset_index()
    .rename(columns={"<lambda_0>": "std"})
)

df_results_to_save.to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>

In [27]:
df_results_to_save

Category     Company  Data Shape Timesteps                  Features  \
                                                                             
0  Local Model  partner_ii  (1053, 14)       NaN  Chemical + Mineralogical   
1  Local Model  partner_ii  (1053, 14)       NaN  Chemical + Mineralogical   
2  Local Model  partner_ii  (1053, 14)       NaN  Chemical + Mineralogical   
3  Local Model  partner_ii  (1053, 14)       NaN  Chemical + Mineralogical   

               Model            Cross Validation  \
                                                   
0  Linear Regression  Blocking Time Series Split   
1  Linear Regression           Out of time Split   
2  Linear Regression              Repeated KFold   
3  Linear Regression           Time Series Split   

                            Cross Validation Params RMSE Train            ...  \
                                                          mean       std  ...   
0  {"N_Splits": 5, "Repeats": 1, "train_size": 0.8}   1.919970  0.181775  ...   
1                                {"Test Size": 0.2}   2.303569  0.000000  ...   
2                     {"N_Splits": 5, "Repeats": 3}   2.409957  0.046638  ...   
3           {"N_Splits": 5, "Repeats": 1, "Gap": 0}   2.234316  0.119416  ...   

   R2 Train           RMSE Test            MAE Test           MAPE Test  \
       mean       std      mean       std      mean       std      mean   
0  0.615678  0.075370  3.312442  1.016668  2.512650  0.680570  0.057328   
1  0.500264  0.000000  3.836344  0.000000  3.253284  0.000000  0.083310   
2  0.533155  0.013587  2.516459  0.234451  1.938508  0.110079  0.045198   
3  0.552439  0.067050  3.038624  0.705377  2.408534  0.631157  0.057276   

              R2 Test            
        std      mean       std  
0  0.013361 -0.114735  0.475081  
1  0.000000 -0.899749  0.000000  
2  0.002606  0.483923  0.078293  
3  0.017876 -0.101039  0.590346  

[4 rows x 24 columns]

<IPython.core.display.Javascript object>

In [28]:
c = pd.read_csv(
    "../../../../../../reports/results/local_models/partner_i-oficial/all_cements/grouped/linear_regression_results_grouped_6.csv",
    header=[0, 1],
).rename(columns=lambda x: "" if "Unnamed" in x else x, level=1)

<IPython.core.display.Javascript object>

In [29]:
c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 24 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   (Category, )                 4 non-null      object 
 1   (Company, )                  4 non-null      object 
 2   (Data Shape, )               4 non-null      object 
 3   (Timesteps, )                0 non-null      float64
 4   (Features, )                 4 non-null      object 
 5   (Model, )                    4 non-null      object 
 6   (Cross Validation, )         4 non-null      object 
 7   (Cross Validation Params, )  4 non-null      object 
 8   (RMSE Train, mean)           4 non-null      float64
 9   (RMSE Train, std)            4 non-null      float64
 10  (MAE Train, mean)            4 non-null      float64
 11  (MAE Train, std)             4 non-null      float64
 12  (MAPE Train, mean)           4 non-null      float64
 13  (MAPE Train, std)       

<IPython.core.display.Javascript object>

In [30]:
df_results_to_save.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 24 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   (Category, )                 4 non-null      object 
 1   (Company, )                  4 non-null      object 
 2   (Data Shape, )               4 non-null      object 
 3   (Timesteps, )                0 non-null      float64
 4   (Features, )                 4 non-null      object 
 5   (Model, )                    4 non-null      object 
 6   (Cross Validation, )         4 non-null      object 
 7   (Cross Validation Params, )  4 non-null      object 
 8   (RMSE Train, mean)           4 non-null      float64
 9   (RMSE Train, std)            4 non-null      float64
 10  (MAE Train, mean)            4 non-null      float64
 11  (MAE Train, std)             4 non-null      float64
 12  (MAPE Train, mean)           4 non-null      float64
 13  (MAPE Train, std)       

<IPython.core.display.Javascript object>

In [31]:
df_results_to_save[df_results_to_save["Cross Validation Params"].str.contains("Date")]

Empty DataFrame
Columns: [(Category, ), (Company, ), (Data Shape, ), (Timesteps, ), (Features, ), (Model, ), (Cross Validation, ), (Cross Validation Params, ), (RMSE Train, mean), (RMSE Train, std), (MAE Train, mean), (MAE Train, std), (MAPE Train, mean), (MAPE Train, std), (R2 Train, mean), (R2 Train, std), (RMSE Test, mean), (RMSE Test, std), (MAE Test, mean), (MAE Test, std), (MAPE Test, mean), (MAPE Test, std), (R2 Test, mean), (R2 Test, std)]
Index: []

[0 rows x 24 columns]

<IPython.core.display.Javascript object>

In [32]:
df.shape

(1053, 24)

<IPython.core.display.Javascript object>

In [33]:
c = pd.read_csv(
    "../../../../../../reports/results/local_models/partner_i-oficial/all_cements/grouped/linear_regression_results_grouped_5.csv",
    header=[0, 1],
).rename(columns=lambda x: "" if "Unnamed" in x else x, level=1)

<IPython.core.display.Javascript object>

In [34]:
c

Category     Company  Data Shape Timesteps  \
                                                   
0  Local Model  partner_i  (1234, 35)       NaN   
1  Local Model  partner_i  (1234, 35)       NaN   
2  Local Model  partner_i  (1234, 35)       NaN   
3  Local Model  partner_i  (1234, 35)       NaN   

                                         Features              Model  \
                                                                       
0  Chemical + Mineralogical + CS3 + CS7 + One-Hot  Linear Regression   
1  Chemical + Mineralogical + CS3 + CS7 + One-Hot  Linear Regression   
2  Chemical + Mineralogical + CS3 + CS7 + One-Hot  Linear Regression   
3  Chemical + Mineralogical + CS3 + CS7 + One-Hot  Linear Regression   

             Cross Validation  \
                                
0  Blocking Time Series Split   
1           Out of time Split   
2              Repeated KFold   
3           Time Series Split   

                            Cross Validation Params RMSE Train            ...  \
                                                          mean       std  ...   
0  {"N_Splits": 5, "Repeats": 1, "train_size": 0.8}   0.727839  0.044848  ...   
1                                {"Test Size": 0.2}   0.820696  0.000000  ...   
2                     {"N_Splits": 5, "Repeats": 3}   0.858495  0.005642  ...   
3           {"N_Splits": 5, "Repeats": 1, "Gap": 0}   0.754234  0.046800  ...   

   R2 Train           RMSE Test            MAE Test           MAPE Test  \
       mean       std      mean       std      mean       std      mean   
0  0.979190  0.001583  0.923776  0.131192  0.719652  0.113011  0.016722   
1  0.973586  0.000000  1.183654  0.000000  0.884513  0.000000  0.019938   
2  0.970967  0.000430  0.906593  0.038163  0.708813  0.023779  0.016380   
3  0.976645  0.002176  1.020291  0.103525  0.778093  0.104477  0.017842   

              R2 Test            
        std      mean       std  
0  0.002862  0.963234  0.014000  
1  0.000000  0.943272  0.000000  
2  0.000502  0.967432  0.002737  
3  0.002215  0.958604  0.009382  

[4 rows x 24 columns]

<IPython.core.display.Javascript object>